In [ ]:
import pathlib
import numpy as np
import skimage
import skimage.measure
import skimage.io
import matplotlib.pyplot as plt

In [ ]:
root = pathlib.Path('/media/ryan/TOSHIBA EXT/Research/datasets/uhcs-segment/images/spheroidite/')
annotations = sorted(list(root.glob('annotation*')))
micrographs = sorted(list(root.glob('micrograph*')))



In [ ]:
len(annotations), len(micrographs)

In [ ]:
annotations[0], micrographs[0]

In [ ]:
i = 1

a1 = skimage.io.imread(annotations[i])
i1 = skimage.io.imread(micrographs[i])

fig, ax = plt.subplots(1,2, sharey=True, figsize=(20,10), dpi=300)
ax[0].imshow(i1, cmap='gray')
ax[1].imshow(a1, cmap='jet')

In [ ]:
labels = skimage.measure.label(a1.astype(np.bool))

In [ ]:
plt.imshow(labels, cmap='jet')

In [ ]:
np.unique(labels)

In [ ]:
img[0,0], img1[0,0]

In [ ]:
import mrcnn_utils
mrcnn_utils.extract_bboxes((labels == 1)[:,:,np.newaxis])[0]

In [ ]:
BoxMode.XYXY_ABS

In [ ]:
from detectron2.structures import BoxMode, Instances
from detectron2.data.detection_utils import annotations_to_instances
import torch

def load_labels(img_path, label_path, dclass):
    """
    img_path: path to original image
    label_path: path to annotation image
    dclass: 'Training,' 'Validation', 'Testing', etc
    """ 

    
    # binary image- 1 for spheroidite, 0 for background
    im = skimage.io.imread(label_path)
    
    r, c = im.shape
    img = skimage.io.imread(img_path, as_gray=True)
    img = skimage.color.gray2rgb(img)[:r,:c,...]
    
    
    ddict = {'file_name': pathlib.Path(img_path),
             'annotation_file': pathlib.Path(label_path),
             'height': r,
             'width': c,
             'dataset_class': dclass,
             'mask_format': "bitmask"}
    
    return ddict
    
    





def mapper(ddict):
    ext_mapper = {'.tiff':255,
                 '.png':0}

    im_b = skimage.io.imread(ddict['annotation_file']) == ext_mapper[ddict['annotation_file'].suffix]
    
    img = skimage.io.imread(ddict['file_name'], as_gray=True)[:ddict['height'], :ddict['width']]
    img = skimage.color.gray2rgb(img)
    
    labels = skimage.measure.label(im_b)
    unique = np.unique(labels)[1:] # ignore 0 pixels 
    
    n = unique.shape[0]
    
    annotations=[]
    for i in range(1,n+1):
        mask = labels == i
        bbox = mrcnn_utils.extract_bboxes(mask[:,:,np.newaxis])[0]
        label = 0
        annotations.append({'bbox': bbox,
                            'bbox_mode': BoxMode.XYXY_ABS,
                            'segmentation': mask,
                            'category_id': label})
    
    instances = annotations_to_instances(annotations, img.shape[:2], mask_format='bitmask')
    
    dataset_dict = {}
    for k, v in ddict.items():
        dataset_dict[k] = v
    # Pytorch's dataloader is efficient on torch.Tensor due to shared-memory,
    # but not efficient on large generic data structures due to the use of pickle & mp.Queue.
    # Therefore it's important to use torch.Tensor.
    dataset_dict["image"] = torch.as_tensor(np.ascontiguousarray(img.transpose(2, 0, 1)))
    dataset_dict['instances'] = instances
    
    
    return dataset_dict

    
    
    
    
    
    
    

In [ ]:
x = load_labels(micrographs[0], annotations[0], 'Train')
y = mapper(x)

In [ ]:
y['instances']

In [ ]:
i = 0
load_labels(micrographs[i], annotations[i])

In [ ]:
for a in annotations:
    load_labels(a)

In [ ]:
i = 0
img = skimage.io.imread(annotations[i])


plt.imshow(img==0, cmap='gray')
plt.colorbar()
plt.title(annotations[i].name)

In [ ]:
for a in annotations:
    print(a.suffix)

In [ ]:
annotations[0].name, annotations[1].name

In [ ]:
i = 1
img1 = skimage.io.imread(annotations[i])
plt.imshow(img1==255, cmap='gray')
plt.colorbar()
plt.title(annotations[i].name)

In [ ]:
for img, ann in zip(micrographs, annotations):
    
    a1 = skimage.io.imread(ann)
    i1 = skimage.io.imread(img)

    
    fig, ax = plt.subplots(1,2, sharey=True, figsize=(20,10), dpi=300)
    labels = skimage.measure.label(a1==0)
    ax[0].imshow(i1, cmap='gray')
    ax[1].imshow(a1, cmap='gray')
    plt.show()
    print(len(np.unique(labels.ravel())))
    